# Antes de iniciar...

- Debe existir una carpeta llamada "optimizacion" y en ella los archivos resultantes de la ejecución anterior, donde el nombre del archivo tiene el id de la sede. Ej: 2161.xlsx

- El archivo de tablas de WFS se debe llamar "TablasWFS.xlsx"

- El archivo de profesionales se debe llamar "2_PROFESIONALES.xlsx"

- La tabla de homologación se debe llamar "stations.csv" y debe estar separado por ";"

---
# Importar librerias necesarias
---

In [1]:
import pandas as pd
import numpy as np
import glob
from datetime import datetime

---
# Definir variables comunes
---

In [2]:
#lista_dias = ['DOMINGO', 'LUNES', 'MARTES', 'MIERCOLES', 'JUEVES', 'VIERNES', 'SABADO', 'FESTIVO']
lista_dias = ['DOMINGO', 'LUNES', 'MARTES', 'MIÉRCOLES', 'JUEVES', 'VIERNES', 'SÁBADO', 'FESTIVO'] # solo desarrollo
lista_dias_wfs = ['Sun', 'Mon', 'Tues', 'Wed', 'Thu', 'Fri', 'Sat', 'Hol']

---
# Leer los archivos
---


## Resultado del modelo

### Leer

In [3]:
# por ahora solo hay uno, pero la idea es leer que archivos estan en la carpeta, luego leer cada uno y agregarlo al mismo dataframe
# una idea es poner el id de la sede como nombre del archivo y asignarlo cada vez que se lea un archivo
# df_modelo = pd.read_excel('/content/Solution_SedeIndustriales.xlsx', 'Hoja1',
#                           usecols = ['persona', 'consultorio', 'turno', 'dia'])
# df_modelo['id_sede'] = '2161'





# se leen todos los archivos de la carpeta "optimizacion"
# el nombre debe ser el id de la sede, ej: 2161.xlsx (industriales)
archivos_xlsx = glob.glob('/content/optimizacion/*.xlsx')

l_dataframes = []
for archivo in archivos_xlsx:
    df_tempi = pd.read_excel(archivo, 'Hoja1') # Hoja1 que tiene el resultado
    nombre_archivo = archivo.split('/')[-1].split('.')[0] # porque el nombre del archivo es el id de la sede
    df_tempi['id_sede'] = nombre_archivo
    l_dataframes.append(df_tempi)

# Combinar los DataFrames en un solo DataFrame utilizando pd.concat()
df_modelo = pd.concat(l_dataframes, ignore_index = True)


df_modelo['persona'] = df_modelo['persona'].astype(str)
df_modelo


,Unnamed: 0,persona,consultorio,turno,dia,valor,id_sede
0,0,1026146052,SILLA_ODONTO_05,T7_13,LUNES,1,2161
1,1,1026146052,SILLA_ODONTO_05,T10_6,LUNES 2,1,2161
2,2,1026146052,SILLA_ODONTO_08,T10_6,JUEVES 2,1,2161
3,3,1026146052,SILLA_ODONTO_08,T10_6,MIÉRCOLES,1,2161
4,4,1026146052,CONS_ODONTO_14,T7_9,MARTES,1,2161
...,...,...,...,...,...,...,...
958,958,71623547,CONS_01_P2,T2_11,MARTES 2,1,2161
959,959,71623547,CONS_01_P2,T3_7,MARTES 2,1,2161
960,960,1128406675,CONS_06_P4,T7_7,MARTES 2,1,2161
961,961,1128406675,CONS_06_P4,T10_7,MARTES,1,2161


### Convertir el turno en hora ini y hora fin

In [4]:
# Convertir el turno en hora ini y hora fin

# se divide
df_modelo[['num_horas', 'hora_ini']] = df_modelo['turno'].str.split('_', expand = True)

# reemplazar la t
df_modelo['num_horas'] = df_modelo['num_horas'].str.replace('T', '').astype(int)

# arreglar tipos de dato
df_modelo['hora_ini'] = df_modelo['hora_ini'].astype(int)
df_modelo['hora_fin'] = df_modelo['hora_ini'] + df_modelo['num_horas']

# ya no lo necesitamos mas
df_modelo = df_modelo.drop('turno', axis = 1)

df_modelo

,Unnamed: 0,persona,consultorio,dia,valor,id_sede,num_horas,hora_ini,hora_fin
0,0,1026146052,SILLA_ODONTO_05,LUNES,1,2161,7,13,20
1,1,1026146052,SILLA_ODONTO_05,LUNES 2,1,2161,10,6,16
2,2,1026146052,SILLA_ODONTO_08,JUEVES 2,1,2161,10,6,16
3,3,1026146052,SILLA_ODONTO_08,MIÉRCOLES,1,2161,10,6,16
4,4,1026146052,CONS_ODONTO_14,MARTES,1,2161,7,9,16
...,...,...,...,...,...,...,...,...,...
958,958,71623547,CONS_01_P2,MARTES 2,1,2161,2,11,13
959,959,71623547,CONS_01_P2,MARTES 2,1,2161,3,7,10
960,960,1128406675,CONS_06_P4,MARTES 2,1,2161,7,7,14
961,961,1128406675,CONS_06_P4,MARTES,1,2161,10,7,17


## Maestro de profesionales

In [5]:
#df_profesionales = pd.read_excel('/content/2_PROFESIONALES.xlsx', 'MAESTRA',
#                          usecols = ['Id_Sede', 'Codigo Sf (Id Profesional)', 'Qualification Cargo (Id Qualification)', 'Station Name '])

# solo desarrollo
df_profesionales = pd.read_excel('/content/2_PROFESIONALES.xlsx', 'MAESTRA',
                          usecols = ['Id_Sede', 'Id_Compañia', 'Nro De Identificación', 'Qualification Cargo (Id Qualification)', 'Station Name '])


df_profesionales = df_profesionales.rename(
    columns = {
        'Id_Sede' : 'id_sede',
        'Id_Compañia' : 'id_tipo_sede',
        #'Codigo Sf (Id Profesional)' : 'id_persona',
        'Nro De Identificación' : 'id_persona', # solo desarrollo
        'Qualification Cargo (Id Qualification)' : 'cualificacion',
        'Station Name ' : 'estacion'
    }
)

df_profesionales['id_sede'] = df_profesionales['id_sede'].astype(str).str.strip() # porque si ya era un string podria tener espacios
df_profesionales['id_persona'] = df_profesionales['id_persona'].astype(str).str.strip() # porque si ya era un string podria tener espacios

df_profesionales



,id_tipo_sede,id_sede,id_persona,cualificacion,estacion
0,22,80,1117532435,IPS_ODONT_GRAL,CENT_ODON3
1,22,80,98548128,IPS_ODONT_GRAL,CENT_ODON3
2,22,80,1110531421,IPS_ODONT_GRAL,CENT_ODON3
3,22,80,32140934,IPS_ODONT_GRAL,CENT_ODON3
4,22,80,1152187766,IPS_ODONT_GRAL,CENT_ODON3
...,...,...,...,...,...
637,28,900247,1098630538,AY_DX_MED_FISIATRA,282J_MD_FIS_CL_RHB
638,28,900789,1128416529,AY_DX_MED_FISIATRA,285K_MED_FIS_BELLO
639,28,900247,1128416529,AY_DX_MED_FISIATRA,285K_MED_FIS_BELLO
640,28,900101,1143224129,AY_DX_MED_RADIOLOGO_I,2809_MED_RAD_I_S_IND


## Tabla homologación:
Station / SchedulingGroup / SchedulingUnit

In [6]:
df_homologacion = pd.read_csv('/content/stations.csv', delimiter = ';',
                              usecols = ['StationName', 'SchedulingGroup', 'SchedulingUnit'])
df_homologacion

,StationName,SchedulingGroup,SchedulingUnit
0,2801_ANALIST_ATM_ANT,ANTIOQUIA_AY_DX,ANTIOQUIA_AY_DX_2801_DIR_REG_ANTIO_ATM_LIDER
1,2801_AUX_ASI_RUT_ANT,ANTIOQUIA_AY_DX,ANTIOQUIA_AY_DX_2801_DIR_REG_ANTIO_ATM_ADMIN
2,2801_AUX_TRA_MUE_ANT,ANTIOQUIA_AY_DX,ANTIOQUIA_AY_DX_2801_DIR_REG_ANTIO_ATM_LAB
3,2801_MENSAJ_ANT,ANTIOQUIA_AY_DX,ANTIOQUIA_AY_DX_2801_DIR_REG_ANTIO_ATM_LAB
4,2802_ANALIST_BIOLO,ANTIOQUIA_AY_DX,ANTIOQUIA_AY_DX_2802_CENTRAL_PROCES_INDUS_BIO_...
...,...,...,...
2136,VIV_BAR_IMA_ELP_D,NORTE_AY_DX,NORTE_AY_DX_283H_VIVA_BARRANQ_IMAG_ELP
2137,VIV_BAR_IM_ELP_TEC_D,NORTE_AY_DX,NORTE_AY_DX_283H_VIVA_BARRANQ_IMAG_ELP_TECN
2138,VIV_BAR_LAB_D,NORTE_AY_DX,NORTE_AY_DX_283H_VIVA_BARRANQ_LAB
2139,VIV_BAR_LAB_IMA_D,NORTE_AY_DX,NORTE_AY_DX_283H_VIVA_BARRANQ_LAB_IMAG


## Tablas de validacion (WFS)

In [7]:
df_wfs = pd.read_excel('/content/TablasWFS.xlsx', header = 1,
                         usecols = ['Station Name (Caracteres 20)', 'Shift Name', 'Qualification Name (40)'])

v_estaciones = df_wfs['Station Name (Caracteres 20)'].dropna().drop_duplicates()#.to_list()
v_turnos = df_wfs['Shift Name'].dropna().drop_duplicates()#.to_list()
v_cualificaciones = df_wfs['Qualification Name (40)'].dropna().drop_duplicates()#.to_list()



---
# Agrupar
---

## (x) Agrupar cuando el horario es continuo

In [7]:
# (Pendiente eliminar) Agrupar cuando el horario es continuo (teniendo en cuenta: persona y el dia, ignorar el consultorio)


## Obtener la hora maxima y minima para cada persona y dia

In [8]:
# (Nuevo) Obtener la hora maxima y minima para cada persona y dia
# No importa si el horario es continuo o no, igual se entrega la franja completa

df_modelo_gp = df_modelo.groupby(['id_sede', 'persona', 'dia']).agg({'hora_ini' : 'min', 'hora_fin' : 'max'})
df_modelo_gp = df_modelo_gp.reset_index()

df_modelo_gp



,id_sede,persona,dia,hora_ini,hora_fin
0,2161,1017133964,JUEVES,13,18
1,2161,1017133964,JUEVES 2,13,18
2,2161,1017133964,MARTES,13,18
3,2161,1017133964,MARTES 2,13,18
4,2161,1017133964,MIÉRCOLES,8,18
...,...,...,...,...,...
783,2161,98663759,MIÉRCOLES 2,7,13
784,2161,98771590,JUEVES,7,12
785,2161,98771590,JUEVES 2,7,12
786,2161,98771590,MIÉRCOLES,14,20


---
# Completar la información
---
Se debe hacer en este punto para poder construir los archivos de patrones de rotación


## Cruzar con el maestro de profesionales

In [9]:
# Traer los campos para formar el archivo plano: con la persona traer el station y qualification del maestro #2 (profesionales)

df_modelo_detallado = pd.merge(df_modelo_gp, df_profesionales, how = 'left', left_on=['id_sede', 'persona'], right_on=['id_sede', 'id_persona'])
df_modelo_detallado = df_modelo_detallado.drop('id_persona', axis = 1)
df_modelo_detallado


,id_sede,persona,dia,hora_ini,hora_fin,id_tipo_sede,cualificacion,estacion
0,2161,1017133964,JUEVES,13,18,22,IPS_ORTOPED,SS_INDU_ORTO5
1,2161,1017133964,JUEVES 2,13,18,22,IPS_ORTOPED,SS_INDU_ORTO5
2,2161,1017133964,MARTES,13,18,22,IPS_ORTOPED,SS_INDU_ORTO5
3,2161,1017133964,MARTES 2,13,18,22,IPS_ORTOPED,SS_INDU_ORTO5
4,2161,1017133964,MIÉRCOLES,8,18,22,IPS_ORTOPED,SS_INDU_ORTO5
...,...,...,...,...,...,...,...,...
783,2161,98663759,MIÉRCOLES 2,7,13,22,IPS_MED_DEPORTO,SS_INDU_DEPOR5
784,2161,98771590,JUEVES,7,12,22,IPS_MED_DEPORTO,SS_INDU_DEPOR5
785,2161,98771590,JUEVES 2,7,12,22,IPS_MED_DEPORTO,SS_INDU_DEPOR5
786,2161,98771590,MIÉRCOLES,14,20,22,IPS_MED_DEPORTO,SS_INDU_DEPOR5


## Construir shift name

In [10]:
# Construir el shift name: con id_compañia poner el prefijo (22: IPS ; 28: Ay_Dx) + hora ini y hora fin

# shift name

def asignar_prefijo_turno(valor):
    if valor == 22:
        return 'IPS'
    elif valor == 28:
        return 'Ay_Dx'
    else:
        return '???'

# primero asignamos el prefijo
df_modelo_detallado['turno'] = df_modelo_detallado['id_tipo_sede'].apply(asignar_prefijo_turno)
df_modelo_detallado = df_modelo_detallado.drop('id_tipo_sede', axis = 1) # al tener el prefijo ya no lo necesitamos
df_modelo_detallado



,id_sede,persona,dia,hora_ini,hora_fin,cualificacion,estacion,turno
0,2161,1017133964,JUEVES,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS
1,2161,1017133964,JUEVES 2,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS
2,2161,1017133964,MARTES,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS
3,2161,1017133964,MARTES 2,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS
4,2161,1017133964,MIÉRCOLES,8,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS
...,...,...,...,...,...,...,...,...
783,2161,98663759,MIÉRCOLES 2,7,13,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS
784,2161,98771590,JUEVES,7,12,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS
785,2161,98771590,JUEVES 2,7,12,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS
786,2161,98771590,MIÉRCOLES,14,20,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS


In [11]:
df_modelo_detallado['turno'] = df_modelo_detallado['turno'] + '_' + \
df_modelo_detallado['hora_ini'].astype(str).str.zfill(2) + '00_' + \
df_modelo_detallado['hora_fin'].astype(str).str.zfill(2) + '00'

df_modelo_detallado



,id_sede,persona,dia,hora_ini,hora_fin,cualificacion,estacion,turno
0,2161,1017133964,JUEVES,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_1300_1800
1,2161,1017133964,JUEVES 2,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_1300_1800
2,2161,1017133964,MARTES,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_1300_1800
3,2161,1017133964,MARTES 2,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_1300_1800
4,2161,1017133964,MIÉRCOLES,8,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_0800_1800
...,...,...,...,...,...,...,...,...
783,2161,98663759,MIÉRCOLES 2,7,13,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_0700_1300
784,2161,98771590,JUEVES,7,12,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_0700_1200
785,2161,98771590,JUEVES 2,7,12,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_0700_1200
786,2161,98771590,MIÉRCOLES,14,20,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_1400_2000


## Construir model name

In [12]:
# Agregar un nombre que haga unico cada registro

df_modelo_detallado['nombre_modelo'] = df_modelo_detallado['estacion'] + '_' + df_modelo_detallado['cualificacion'] + '_' + df_modelo_detallado['turno']
df_modelo_detallado


,id_sede,persona,dia,hora_ini,hora_fin,cualificacion,estacion,turno,nombre_modelo
0,2161,1017133964,JUEVES,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_1300_1800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_1300_1800
1,2161,1017133964,JUEVES 2,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_1300_1800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_1300_1800
2,2161,1017133964,MARTES,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_1300_1800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_1300_1800
3,2161,1017133964,MARTES 2,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_1300_1800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_1300_1800
4,2161,1017133964,MIÉRCOLES,8,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_0800_1800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_0800_1800
...,...,...,...,...,...,...,...,...,...
783,2161,98663759,MIÉRCOLES 2,7,13,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_0700_1300,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_0700_1300
784,2161,98771590,JUEVES,7,12,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_0700_1200,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_0700_1200
785,2161,98771590,JUEVES 2,7,12,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_0700_1200,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_0700_1200
786,2161,98771590,MIÉRCOLES,14,20,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_1400_2000,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_1400_2000


---
# Construir dfs de patrones de rotación : df_maestro_patrones y df_detalle_patrones
---

## Detalle rotación
Pasar los dias (texto) al numero que representa
1: Lunes - 2: Martes - ... - 7: Domingo - 8: Lunes 2 - 9: Martes 2 - ...

In [13]:
# Todo menos el ultimo elemento: "festivo"
lista_dias_full = lista_dias[:-1]

# Para que el domingo quede al final:
tempi = lista_dias_full.pop(0)
lista_dias_full.append(tempi)

# Agregar la lista otra vez pero con el "2"
lista_dias_full = lista_dias_full + [dia + " 2" for dia in lista_dias_full]

lista_dias_full



['LUNES',
 'MARTES',
 'MIÉRCOLES',
 'JUEVES',
 'VIERNES',
 'SÁBADO',
 'DOMINGO',
 'LUNES 2',
 'MARTES 2',
 'MIÉRCOLES 2',
 'JUEVES 2',
 'VIERNES 2',
 'SÁBADO 2',
 'DOMINGO 2']

In [14]:
# Pasar dia a numero
df_detalle_patrones = df_modelo_detallado.copy()
df_detalle_patrones['dia'] = df_detalle_patrones['dia'].apply(lambda x: lista_dias_full.index(x) + 1).astype(str)
df_detalle_patrones

,id_sede,persona,dia,hora_ini,hora_fin,cualificacion,estacion,turno,nombre_modelo
0,2161,1017133964,4,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_1300_1800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_1300_1800
1,2161,1017133964,11,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_1300_1800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_1300_1800
2,2161,1017133964,2,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_1300_1800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_1300_1800
3,2161,1017133964,9,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_1300_1800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_1300_1800
4,2161,1017133964,3,8,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_0800_1800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_0800_1800
...,...,...,...,...,...,...,...,...,...
783,2161,98663759,10,7,13,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_0700_1300,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_0700_1300
784,2161,98771590,4,7,12,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_0700_1200,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_0700_1200
785,2161,98771590,11,7,12,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_0700_1200,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_0700_1200
786,2161,98771590,3,14,20,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_1400_2000,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_1400_2000


In [15]:
df_detalle_patrones['nombre_patron'] = df_detalle_patrones['persona'] + '_' + df_detalle_patrones['estacion']
df_detalle_patrones


,id_sede,persona,dia,hora_ini,hora_fin,cualificacion,estacion,turno,nombre_modelo,nombre_patron
0,2161,1017133964,4,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_1300_1800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_1300_1800,1017133964_SS_INDU_ORTO5
1,2161,1017133964,11,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_1300_1800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_1300_1800,1017133964_SS_INDU_ORTO5
2,2161,1017133964,2,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_1300_1800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_1300_1800,1017133964_SS_INDU_ORTO5
3,2161,1017133964,9,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_1300_1800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_1300_1800,1017133964_SS_INDU_ORTO5
4,2161,1017133964,3,8,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_0800_1800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_0800_1800,1017133964_SS_INDU_ORTO5
...,...,...,...,...,...,...,...,...,...,...
783,2161,98663759,10,7,13,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_0700_1300,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_0700_1300,98663759_SS_INDU_DEPOR5
784,2161,98771590,4,7,12,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_0700_1200,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_0700_1200,98771590_SS_INDU_DEPOR5
785,2161,98771590,11,7,12,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_0700_1200,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_0700_1200,98771590_SS_INDU_DEPOR5
786,2161,98771590,3,14,20,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_1400_2000,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_1400_2000,98771590_SS_INDU_DEPOR5


## Maestro de rotación

In [16]:
df_maestro_patrones = df_detalle_patrones[['nombre_patron', 'estacion']].drop_duplicates()
df_maestro_patrones['detalle'] = df_maestro_patrones['nombre_patron']
df_maestro_patrones

,nombre_patron,estacion,detalle
0,1017133964_SS_INDU_ORTO5,SS_INDU_ORTO5,1017133964_SS_INDU_ORTO5
6,1026146052_SS_INDU_ODON3,SS_INDU_ODON3,1026146052_SS_INDU_ODON3
13,1036623713_SS_INDU_ODON3,SS_INDU_ODON3,1036623713_SS_INDU_ODON3
24,1036635690_SS_INDU_PSI14,SS_INDU_PSI14,1036635690_SS_INDU_PSI14
35,1036646761_SS_INDU_PSI14,SS_INDU_PSI14,1036646761_SS_INDU_PSI14
...,...,...,...
765,98587916_SS_INDU_ODON3,SS_INDU_ODON3,98587916_SS_INDU_ODON3
770,98624874_SS_SAO_ORTO5,SS_SAO_ORTO5,98624874_SS_SAO_ORTO5
772,98639336_SS_INDU_ODON3,SS_INDU_ODON3,98639336_SS_INDU_ODON3
778,98663759_SS_INDU_DEPOR5,SS_INDU_DEPOR5,98663759_SS_INDU_DEPOR5


In [17]:
# Completar informacion con la tabla de homologacion
df_maestro_patrones = pd.merge(df_maestro_patrones, df_homologacion, how = 'left', left_on = 'estacion', right_on = 'StationName')
df_maestro_patrones = df_maestro_patrones[['nombre_patron', 'detalle', 'SchedulingGroup', 'SchedulingUnit']]

df_maestro_patrones

,nombre_patron,detalle,SchedulingGroup,SchedulingUnit
0,1017133964_SS_INDU_ORTO5,1017133964_SS_INDU_ORTO5,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_INDUSTRIALES_MED_ESP
1,1026146052_SS_INDU_ODON3,1026146052_SS_INDU_ODON3,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_INDUSTRIALES_ODONTO
2,1036623713_SS_INDU_ODON3,1036623713_SS_INDU_ODON3,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_INDUSTRIALES_ODONTO
3,1036635690_SS_INDU_PSI14,1036635690_SS_INDU_PSI14,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_INDUSTRIALES_PYP
4,1036646761_SS_INDU_PSI14,1036646761_SS_INDU_PSI14,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_INDUSTRIALES_PYP
...,...,...,...,...
112,98587916_SS_INDU_ODON3,98587916_SS_INDU_ODON3,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_INDUSTRIALES_ODONTO
113,98624874_SS_SAO_ORTO5,98624874_SS_SAO_ORTO5,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_SAO_PAULO_MED_ESP
114,98639336_SS_INDU_ODON3,98639336_SS_INDU_ODON3,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_INDUSTRIALES_ODONTO
115,98663759_SS_INDU_DEPOR5,98663759_SS_INDU_DEPOR5,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_INDUSTRIALES_MED_ESP


## Continuar con el detalle

In [18]:
df_detalle_patrones['esta_trabajando'] = 'Y'
df_detalle_patrones = df_detalle_patrones[['nombre_patron', 'dia', 'esta_trabajando', 'nombre_modelo']]
df_detalle_patrones

,nombre_patron,dia,esta_trabajando,nombre_modelo
0,1017133964_SS_INDU_ORTO5,4,Y,SS_INDU_ORTO5_IPS_ORTOPED_IPS_1300_1800
1,1017133964_SS_INDU_ORTO5,11,Y,SS_INDU_ORTO5_IPS_ORTOPED_IPS_1300_1800
2,1017133964_SS_INDU_ORTO5,2,Y,SS_INDU_ORTO5_IPS_ORTOPED_IPS_1300_1800
3,1017133964_SS_INDU_ORTO5,9,Y,SS_INDU_ORTO5_IPS_ORTOPED_IPS_1300_1800
4,1017133964_SS_INDU_ORTO5,3,Y,SS_INDU_ORTO5_IPS_ORTOPED_IPS_0800_1800
...,...,...,...,...
783,98663759_SS_INDU_DEPOR5,10,Y,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_0700_1300
784,98771590_SS_INDU_DEPOR5,4,Y,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_0700_1200
785,98771590_SS_INDU_DEPOR5,11,Y,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_0700_1200
786,98771590_SS_INDU_DEPOR5,3,Y,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_1400_2000


In [19]:
patrones = df_detalle_patrones['nombre_patron'].unique()
df_patrones_full = pd.DataFrame({'nombre_patron' : [], 'dia' : []})

for i in range(1, 15):
  _df = pd.DataFrame({'nombre_patron' : patrones})
  _df['dia'] = f"{i}"
  df_patrones_full = pd.concat([df_patrones_full, _df])

df_patrones_full

,nombre_patron,dia
0,1017133964_SS_INDU_ORTO5,1
1,1026146052_SS_INDU_ODON3,1
2,1036623713_SS_INDU_ODON3,1
3,1036635690_SS_INDU_PSI14,1
4,1036646761_SS_INDU_PSI14,1
...,...,...
112,98587916_SS_INDU_ODON3,14
113,98624874_SS_SAO_ORTO5,14
114,98639336_SS_INDU_ODON3,14
115,98663759_SS_INDU_DEPOR5,14


In [20]:
df_detalle_patrones = pd.merge(df_patrones_full, df_detalle_patrones, how = 'outer', on = ['nombre_patron', 'dia'])
df_detalle_patrones['esta_trabajando'] = df_detalle_patrones['esta_trabajando'].fillna('Y')
df_detalle_patrones['nombre_modelo'] = df_detalle_patrones['nombre_modelo'].fillna('DESCANSO')

df_detalle_patrones

,nombre_patron,dia,esta_trabajando,nombre_modelo
0,1017133964_SS_INDU_ORTO5,1,Y,DESCANSO
1,1026146052_SS_INDU_ODON3,1,Y,SS_INDU_ODON3_IPS_ODONT_GRAL_IPS_1300_2000
2,1036623713_SS_INDU_ODON3,1,Y,SS_INDU_ODON3_IPS_ODONT_GRAL_IPS_1500_2000
3,1036635690_SS_INDU_PSI14,1,Y,SS_INDU_PSI14_IPS_PSICOLOGIA_IPS_0600_1300
4,1036646761_SS_INDU_PSI14,1,Y,SS_INDU_PSI14_IPS_PSICOLOGIA_IPS_0600_1600
...,...,...,...,...
1633,98587916_SS_INDU_ODON3,14,Y,DESCANSO
1634,98624874_SS_SAO_ORTO5,14,Y,DESCANSO
1635,98639336_SS_INDU_ODON3,14,Y,DESCANSO
1636,98663759_SS_INDU_DEPOR5,14,Y,DESCANSO


---
# Continuar con la creación del df de modelos
---

## Quitar el "2" en los dias y agrupar teniendo en cuenta: persona, horario (tiempos ini y fin) y dia (ignoramos consultorio)

In [21]:
# Quitar el "2" en los dias
df_modelo_detallado['dia'] = df_modelo_detallado['dia'].str.replace(' 2', '')
df_modelo_detallado

,id_sede,persona,dia,hora_ini,hora_fin,cualificacion,estacion,turno,nombre_modelo
0,2161,1017133964,JUEVES,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_1300_1800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_1300_1800
1,2161,1017133964,JUEVES,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_1300_1800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_1300_1800
2,2161,1017133964,MARTES,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_1300_1800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_1300_1800
3,2161,1017133964,MARTES,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_1300_1800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_1300_1800
4,2161,1017133964,MIÉRCOLES,8,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_0800_1800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_0800_1800
...,...,...,...,...,...,...,...,...,...
783,2161,98663759,MIÉRCOLES,7,13,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_0700_1300,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_0700_1300
784,2161,98771590,JUEVES,7,12,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_0700_1200,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_0700_1200
785,2161,98771590,JUEVES,7,12,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_0700_1200,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_0700_1200
786,2161,98771590,MIÉRCOLES,14,20,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_1400_2000,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_1400_2000


## Agrupar teniendo en cuenta: persona, horario (tiempos ini y fin) y dia

---
\+ todos los cambios nuevos: estacion, cualificacion, turno y nombre_modelo


In [22]:
# Agrupar teniendo en cuenta: persona, horario (tiempos ini y fin)
# y dia (ignoramos consultorio)

# agrupamos porque si tengo la misma persona en el mismo dia (semanas diferentes), solo me deberia quedar una sola vez
# por otro lado, si tengo la misma persona en el mismo dia y tienen horarios diferentes (semanas diferentes) debo dejar los dos

df_modelo_det_agrup = df_modelo_detallado.drop_duplicates(['id_sede', 'persona', 'dia', 'hora_ini', 'hora_fin',
                                                           'cualificacion', 'estacion', 'turno', 'nombre_modelo'])
df_modelo_det_agrup


,id_sede,persona,dia,hora_ini,hora_fin,cualificacion,estacion,turno,nombre_modelo
0,2161,1017133964,JUEVES,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_1300_1800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_1300_1800
2,2161,1017133964,MARTES,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_1300_1800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_1300_1800
4,2161,1017133964,MIÉRCOLES,8,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_0800_1800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_0800_1800
6,2161,1026146052,JUEVES,7,19,IPS_ODONT_GRAL,SS_INDU_ODON3,IPS_0700_1900,SS_INDU_ODON3_IPS_ODONT_GRAL_IPS_0700_1900
7,2161,1026146052,JUEVES,6,16,IPS_ODONT_GRAL,SS_INDU_ODON3,IPS_0600_1600,SS_INDU_ODON3_IPS_ODONT_GRAL_IPS_0600_1600
...,...,...,...,...,...,...,...,...,...
779,2161,98663759,LUNES,7,19,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_0700_1900,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_0700_1900
780,2161,98663759,MARTES,7,19,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_0700_1900,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_0700_1900
782,2161,98663759,MIÉRCOLES,7,13,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_0700_1300,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_0700_1300
784,2161,98771590,JUEVES,7,12,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_0700_1200,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_0700_1200


## "Transponer" los dias

In [23]:
# "Transponer" los dias
df_modelo_det_agrup = df_modelo_det_agrup.pivot_table(index=['id_sede', 'persona', 'hora_ini', 'hora_fin',
                                                             'cualificacion', 'estacion', 'turno', 'nombre_modelo'],
                                                      columns = 'dia', aggfunc = lambda x: 1)
df_modelo_det_agrup = df_modelo_det_agrup.fillna(0)
df_modelo_det_agrup = df_modelo_det_agrup.reset_index()

# agregar las columnas que hagan falta, si no tenemos registros para un dia
# especifico, no existira la columna despues de transponer
campos_faltantes = set(lista_dias) - set(df_modelo_det_agrup.columns)
for campo in campos_faltantes:
  df_modelo_det_agrup[f'{campo}'] = 0.0

df_modelo_det_agrup

dia,id_sede,persona,hora_ini,hora_fin,cualificacion,estacion,turno,nombre_modelo,JUEVES,LUNES,MARTES,MIÉRCOLES,SÁBADO,VIERNES,DOMINGO,FESTIVO
0,2161,1017133964,8,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_0800_1800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_0800_1800,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2161,1017133964,13,18,IPS_ORTOPED,SS_INDU_ORTO5,IPS_1300_1800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_1300_1800,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2161,1026146052,6,16,IPS_ODONT_GRAL,SS_INDU_ODON3,IPS_0600_1600,SS_INDU_ODON3_IPS_ODONT_GRAL_IPS_0600_1600,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
3,2161,1026146052,7,19,IPS_ODONT_GRAL,SS_INDU_ODON3,IPS_0700_1900,SS_INDU_ODON3_IPS_ODONT_GRAL_IPS_0700_1900,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2161,1026146052,9,16,IPS_ODONT_GRAL,SS_INDU_ODON3,IPS_0900_1600,SS_INDU_ODON3_IPS_ODONT_GRAL_IPS_0900_1600,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408,2161,98663759,7,13,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_0700_1300,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_0700_1300,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
409,2161,98663759,7,18,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_0700_1800,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_0700_1800,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
410,2161,98663759,7,19,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_0700_1900,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_0700_1900,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
411,2161,98771590,7,12,IPS_MED_DEPORTO,SS_INDU_DEPOR5,IPS_0700_1200,SS_INDU_DEPOR5_IPS_MED_DEPORTO_IPS_0700_1200,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


---
# Agregar los campos "calculados"





---

## Agrupar sin tener en cuenta las personas

In [24]:
# Agrupar utilizando StationName, ShiftName, QualificationNames y los ini y fin de cada dia

df_modelos_eu = df_modelo_det_agrup.groupby(['hora_ini', 'hora_fin', 'cualificacion', 'estacion',
                                             'turno', 'nombre_modelo'])[lista_dias].agg(['sum'])
df_modelos_eu = df_modelos_eu.reset_index()

# quitar los "sum"
df_modelos_eu.columns = [sublista[0] for sublista in df_modelos_eu.columns]

df_modelos_eu

,hora_ini,hora_fin,cualificacion,estacion,turno,nombre_modelo,DOMINGO,LUNES,MARTES,MIÉRCOLES,JUEVES,VIERNES,SÁBADO,FESTIVO
0,6,7,IPS_DERMATOLO,SS_INDU_DERMA5,IPS_0600_0700,SS_INDU_DERMA5_IPS_DERMATOLO_IPS_0600_0700,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6,8,IPS_NUTRICION,SS_INDU_NUTRI1,IPS_0600_0800,SS_INDU_NUTRI1_IPS_NUTRICION_IPS_0600_0800,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,6,8,IPS_ORTOPED,SS_INDU_ORTO5,IPS_0600_0800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_0600_0800,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,6,9,IPS_ORTOPED,SS_INDU_ORTO5,IPS_0600_0900,SS_INDU_ORTO5_IPS_ORTOPED_IPS_0600_0900,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,6,11,IPS_ENDO,SS_INDU_ODON3,IPS_0600_1100,SS_INDU_ODON3_IPS_ENDO_IPS_0600_1100,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,19,22,IPS_PEDIATRA,SS_SAO_PEDI5,IPS_1900_2200,SS_SAO_PEDI5_IPS_PEDIATRA_IPS_1900_2200,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
280,20,21,IPS_PEDIATRA,SS_SAO_PEDI5,IPS_2000_2100,SS_SAO_PEDI5_IPS_PEDIATRA_IPS_2000_2100,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
281,20,22,IPS_PEDIATRA,SS_SAO_PEDI5,IPS_2000_2200,SS_SAO_PEDI5_IPS_PEDIATRA_IPS_2000_2200,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
282,21,22,IPS_PEDIATRA,SS_INDU_PEDI5,IPS_2100_2200,SS_INDU_PEDI5_IPS_PEDIATRA_IPS_2100_2200,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Dar formato a la hora


In [25]:
#
df_modelos_eu['hora_ini'] = df_modelos_eu['hora_ini'].astype(str) + ':00'
df_modelos_eu['hora_fin'] = df_modelos_eu['hora_fin'].astype(str) + ':00'

df_modelos_eu

,hora_ini,hora_fin,cualificacion,estacion,turno,nombre_modelo,DOMINGO,LUNES,MARTES,MIÉRCOLES,JUEVES,VIERNES,SÁBADO,FESTIVO
0,6:00,7:00,IPS_DERMATOLO,SS_INDU_DERMA5,IPS_0600_0700,SS_INDU_DERMA5_IPS_DERMATOLO_IPS_0600_0700,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6:00,8:00,IPS_NUTRICION,SS_INDU_NUTRI1,IPS_0600_0800,SS_INDU_NUTRI1_IPS_NUTRICION_IPS_0600_0800,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,6:00,8:00,IPS_ORTOPED,SS_INDU_ORTO5,IPS_0600_0800,SS_INDU_ORTO5_IPS_ORTOPED_IPS_0600_0800,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,6:00,9:00,IPS_ORTOPED,SS_INDU_ORTO5,IPS_0600_0900,SS_INDU_ORTO5_IPS_ORTOPED_IPS_0600_0900,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,6:00,11:00,IPS_ENDO,SS_INDU_ODON3,IPS_0600_1100,SS_INDU_ODON3_IPS_ENDO_IPS_0600_1100,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,19:00,22:00,IPS_PEDIATRA,SS_SAO_PEDI5,IPS_1900_2200,SS_SAO_PEDI5_IPS_PEDIATRA_IPS_1900_2200,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
280,20:00,21:00,IPS_PEDIATRA,SS_SAO_PEDI5,IPS_2000_2100,SS_SAO_PEDI5_IPS_PEDIATRA_IPS_2000_2100,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
281,20:00,22:00,IPS_PEDIATRA,SS_SAO_PEDI5,IPS_2000_2200,SS_SAO_PEDI5_IPS_PEDIATRA_IPS_2000_2200,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
282,21:00,22:00,IPS_PEDIATRA,SS_INDU_PEDI5,IPS_2100_2200,SS_INDU_PEDI5_IPS_PEDIATRA_IPS_2100_2200,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


---
# Renombrar y ordenar (parcial)
---


## Modelos

In [26]:
# renombrar
df_modelos_eu = df_modelos_eu.rename(
    columns = {
        'nombre_modelo' : 'ModelName',
        'estacion' : 'StationName',
        'turno' : 'ShiftName',
        'cualificacion' : 'QualificationNames',
    }
)

# ordenar:
nuevo_orden = ['ModelName', 'StationName', 'ShiftName', 'QualificationNames',
               'hora_ini', 'hora_fin'] + lista_dias
df_modelos_eu = df_modelos_eu.reindex(columns = nuevo_orden, fill_value = None)


df_modelos_eu

,ModelName,StationName,ShiftName,QualificationNames,hora_ini,hora_fin,DOMINGO,LUNES,MARTES,MIÉRCOLES,JUEVES,VIERNES,SÁBADO,FESTIVO
0,SS_INDU_DERMA5_IPS_DERMATOLO_IPS_0600_0700,SS_INDU_DERMA5,IPS_0600_0700,IPS_DERMATOLO,6:00,7:00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,SS_INDU_NUTRI1_IPS_NUTRICION_IPS_0600_0800,SS_INDU_NUTRI1,IPS_0600_0800,IPS_NUTRICION,6:00,8:00,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,SS_INDU_ORTO5_IPS_ORTOPED_IPS_0600_0800,SS_INDU_ORTO5,IPS_0600_0800,IPS_ORTOPED,6:00,8:00,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,SS_INDU_ORTO5_IPS_ORTOPED_IPS_0600_0900,SS_INDU_ORTO5,IPS_0600_0900,IPS_ORTOPED,6:00,9:00,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,SS_INDU_ODON3_IPS_ENDO_IPS_0600_1100,SS_INDU_ODON3,IPS_0600_1100,IPS_ENDO,6:00,11:00,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,SS_SAO_PEDI5_IPS_PEDIATRA_IPS_1900_2200,SS_SAO_PEDI5,IPS_1900_2200,IPS_PEDIATRA,19:00,22:00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
280,SS_SAO_PEDI5_IPS_PEDIATRA_IPS_2000_2100,SS_SAO_PEDI5,IPS_2000_2100,IPS_PEDIATRA,20:00,21:00,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
281,SS_SAO_PEDI5_IPS_PEDIATRA_IPS_2000_2200,SS_SAO_PEDI5,IPS_2000_2200,IPS_PEDIATRA,20:00,22:00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
282,SS_INDU_PEDI5_IPS_PEDIATRA_IPS_2100_2200,SS_INDU_PEDI5,IPS_2100_2200,IPS_PEDIATRA,21:00,22:00,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Maestro de rotación

In [27]:
# renombrar
df_maestro_patrones = df_maestro_patrones.rename(
    columns = {
        'nombre_patron' : 'RotationPatternName',
        'detalle' : 'Description'
    }
)

df_maestro_patrones

,RotationPatternName,Description,SchedulingGroup,SchedulingUnit
0,1017133964_SS_INDU_ORTO5,1017133964_SS_INDU_ORTO5,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_INDUSTRIALES_MED_ESP
1,1026146052_SS_INDU_ODON3,1026146052_SS_INDU_ODON3,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_INDUSTRIALES_ODONTO
2,1036623713_SS_INDU_ODON3,1036623713_SS_INDU_ODON3,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_INDUSTRIALES_ODONTO
3,1036635690_SS_INDU_PSI14,1036635690_SS_INDU_PSI14,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_INDUSTRIALES_PYP
4,1036646761_SS_INDU_PSI14,1036646761_SS_INDU_PSI14,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_INDUSTRIALES_PYP
...,...,...,...,...
112,98587916_SS_INDU_ODON3,98587916_SS_INDU_ODON3,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_INDUSTRIALES_ODONTO
113,98624874_SS_SAO_ORTO5,98624874_SS_SAO_ORTO5,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_SAO_PAULO_MED_ESP
114,98639336_SS_INDU_ODON3,98639336_SS_INDU_ODON3,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_INDUSTRIALES_ODONTO
115,98663759_SS_INDU_DEPOR5,98663759_SS_INDU_DEPOR5,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_INDUSTRIALES_MED_ESP


## Detalle de rotación

In [28]:
# renombrar
df_detalle_patrones = df_detalle_patrones.rename(
    columns = {
        'nombre_patron' : 'RotationPatternName',
        'dia' : 'DayNumber',
        'esta_trabajando' : 'IsWorking',
        'nombre_modelo' : 'Model'
    }
)

df_detalle_patrones

,RotationPatternName,DayNumber,IsWorking,Model
0,1017133964_SS_INDU_ORTO5,1,Y,DESCANSO
1,1026146052_SS_INDU_ODON3,1,Y,SS_INDU_ODON3_IPS_ODONT_GRAL_IPS_1300_2000
2,1036623713_SS_INDU_ODON3,1,Y,SS_INDU_ODON3_IPS_ODONT_GRAL_IPS_1500_2000
3,1036635690_SS_INDU_PSI14,1,Y,SS_INDU_PSI14_IPS_PSICOLOGIA_IPS_0600_1300
4,1036646761_SS_INDU_PSI14,1,Y,SS_INDU_PSI14_IPS_PSICOLOGIA_IPS_0600_1600
...,...,...,...,...
1633,98587916_SS_INDU_ODON3,14,Y,DESCANSO
1634,98624874_SS_SAO_ORTO5,14,Y,DESCANSO
1635,98639336_SS_INDU_ODON3,14,Y,DESCANSO
1636,98663759_SS_INDU_DEPOR5,14,Y,DESCANSO


---
# Agregar campos restantes ("fijos" y vacios)
---


## Modelos

In [29]:

df_modelos_eu['ModelType'] = 'WORK'
df_modelos_eu['StartDate'] = datetime.now().strftime("%Y-%m-%d")
df_modelos_eu['EndDate'] = '3000-12-31'
df_modelos_eu['DifficultyRating'] = 0
df_modelos_eu['AllowJobOverlap'] = 'N'
df_modelos_eu['CalloutOrder'] = 'NOT_REQUIRED'
df_modelos_eu['CalloutWritesOTEqualization'] = 'NOT_REQUIRED'
df_modelos_eu['FMStatus'] = ''
df_modelos_eu['Description'] = df_modelos_eu['ShiftName']
df_modelos_eu['DbsDataPolicy'] = ''
df_modelos_eu['DbsHierarchyNode'] = ''


for num_dia in range(0, 8):

  df_modelos_eu[f'{lista_dias_wfs[num_dia]}Start'] = df_modelos_eu['hora_ini']
  df_modelos_eu[f'{lista_dias_wfs[num_dia]}End'] = df_modelos_eu['hora_fin']

  df_modelos_eu[f'{lista_dias_wfs[num_dia]}Proficiency'] = ''
  df_modelos_eu[f'{lista_dias_wfs[num_dia]}RequiredHeadCount'] = df_modelos_eu[f'{lista_dias[num_dia]}'].astype(int)

  df_modelos_eu[f'{lista_dias_wfs[num_dia]}OptionalHeadCount'] = ''
  df_modelos_eu[f'{lista_dias_wfs[num_dia]}ScheduleOrder'] = ''
  df_modelos_eu[f'{lista_dias_wfs[num_dia]}LaborStandard'] = ''


for i in range(1, 16):
  df_modelos_eu[f'UserField{i}'] = ''

# ya no se necesitan
df_modelos_eu = df_modelos_eu.drop(['hora_ini', 'hora_fin'] + lista_dias, axis = 1)

df_modelos_eu

,ModelName,StationName,ShiftName,QualificationNames,ModelType,StartDate,EndDate,DifficultyRating,AllowJobOverlap,CalloutOrder,...,UserField6,UserField7,UserField8,UserField9,UserField10,UserField11,UserField12,UserField13,UserField14,UserField15
0,SS_INDU_DERMA5_IPS_DERMATOLO_IPS_0600_0700,SS_INDU_DERMA5,IPS_0600_0700,IPS_DERMATOLO,WORK,2023-07-04,3000-12-31,0,N,NOT_REQUIRED,...,,,,,,,,,,
1,SS_INDU_NUTRI1_IPS_NUTRICION_IPS_0600_0800,SS_INDU_NUTRI1,IPS_0600_0800,IPS_NUTRICION,WORK,2023-07-04,3000-12-31,0,N,NOT_REQUIRED,...,,,,,,,,,,
2,SS_INDU_ORTO5_IPS_ORTOPED_IPS_0600_0800,SS_INDU_ORTO5,IPS_0600_0800,IPS_ORTOPED,WORK,2023-07-04,3000-12-31,0,N,NOT_REQUIRED,...,,,,,,,,,,
3,SS_INDU_ORTO5_IPS_ORTOPED_IPS_0600_0900,SS_INDU_ORTO5,IPS_0600_0900,IPS_ORTOPED,WORK,2023-07-04,3000-12-31,0,N,NOT_REQUIRED,...,,,,,,,,,,
4,SS_INDU_ODON3_IPS_ENDO_IPS_0600_1100,SS_INDU_ODON3,IPS_0600_1100,IPS_ENDO,WORK,2023-07-04,3000-12-31,0,N,NOT_REQUIRED,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,SS_SAO_PEDI5_IPS_PEDIATRA_IPS_1900_2200,SS_SAO_PEDI5,IPS_1900_2200,IPS_PEDIATRA,WORK,2023-07-04,3000-12-31,0,N,NOT_REQUIRED,...,,,,,,,,,,
280,SS_SAO_PEDI5_IPS_PEDIATRA_IPS_2000_2100,SS_SAO_PEDI5,IPS_2000_2100,IPS_PEDIATRA,WORK,2023-07-04,3000-12-31,0,N,NOT_REQUIRED,...,,,,,,,,,,
281,SS_SAO_PEDI5_IPS_PEDIATRA_IPS_2000_2200,SS_SAO_PEDI5,IPS_2000_2200,IPS_PEDIATRA,WORK,2023-07-04,3000-12-31,0,N,NOT_REQUIRED,...,,,,,,,,,,
282,SS_INDU_PEDI5_IPS_PEDIATRA_IPS_2100_2200,SS_INDU_PEDI5,IPS_2100_2200,IPS_PEDIATRA,WORK,2023-07-04,3000-12-31,0,N,NOT_REQUIRED,...,,,,,,,,,,


## Maestro de rotación

In [30]:
for i in range(1, 11):
  df_maestro_patrones[f'UserField{i}'] = ''

df_maestro_patrones

,RotationPatternName,Description,SchedulingGroup,SchedulingUnit,UserField1,UserField2,UserField3,UserField4,UserField5,UserField6,UserField7,UserField8,UserField9,UserField10
0,1017133964_SS_INDU_ORTO5,1017133964_SS_INDU_ORTO5,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_INDUSTRIALES_MED_ESP,,,,,,,,,,
1,1026146052_SS_INDU_ODON3,1026146052_SS_INDU_ODON3,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_INDUSTRIALES_ODONTO,,,,,,,,,,
2,1036623713_SS_INDU_ODON3,1036623713_SS_INDU_ODON3,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_INDUSTRIALES_ODONTO,,,,,,,,,,
3,1036635690_SS_INDU_PSI14,1036635690_SS_INDU_PSI14,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_INDUSTRIALES_PYP,,,,,,,,,,
4,1036646761_SS_INDU_PSI14,1036646761_SS_INDU_PSI14,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_INDUSTRIALES_PYP,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,98587916_SS_INDU_ODON3,98587916_SS_INDU_ODON3,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_INDUSTRIALES_ODONTO,,,,,,,,,,
113,98624874_SS_SAO_ORTO5,98624874_SS_SAO_ORTO5,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_SAO_PAULO_MED_ESP,,,,,,,,,,
114,98639336_SS_INDU_ODON3,98639336_SS_INDU_ODON3,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_INDUSTRIALES_ODONTO,,,,,,,,,,
115,98663759_SS_INDU_DEPOR5,98663759_SS_INDU_DEPOR5,ANTIOQUIA_IPS,ANTIOQUIA_IPS_SALUD_SURA_INDUSTRIALES_MED_ESP,,,,,,,,,,


## Detalle de rotación

In [31]:
df_detalle_patrones['ReportNotation'] = ''
df_detalle_patrones['MinBreakBetweenShifts'] = ''

for i in range(1, 6):
  df_detalle_patrones[f'UserField{i}'] = ''

for i in range(1, 6):
  df_detalle_patrones[f'FMDayUserField{i}'] = ''

df_detalle_patrones



,RotationPatternName,DayNumber,IsWorking,Model,ReportNotation,MinBreakBetweenShifts,UserField1,UserField2,UserField3,UserField4,UserField5,FMDayUserField1,FMDayUserField2,FMDayUserField3,FMDayUserField4,FMDayUserField5
0,1017133964_SS_INDU_ORTO5,1,Y,DESCANSO,,,,,,,,,,,,
1,1026146052_SS_INDU_ODON3,1,Y,SS_INDU_ODON3_IPS_ODONT_GRAL_IPS_1300_2000,,,,,,,,,,,,
2,1036623713_SS_INDU_ODON3,1,Y,SS_INDU_ODON3_IPS_ODONT_GRAL_IPS_1500_2000,,,,,,,,,,,,
3,1036635690_SS_INDU_PSI14,1,Y,SS_INDU_PSI14_IPS_PSICOLOGIA_IPS_0600_1300,,,,,,,,,,,,
4,1036646761_SS_INDU_PSI14,1,Y,SS_INDU_PSI14_IPS_PSICOLOGIA_IPS_0600_1600,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1633,98587916_SS_INDU_ODON3,14,Y,DESCANSO,,,,,,,,,,,,
1634,98624874_SS_SAO_ORTO5,14,Y,DESCANSO,,,,,,,,,,,,
1635,98639336_SS_INDU_ODON3,14,Y,DESCANSO,,,,,,,,,,,,
1636,98663759_SS_INDU_DEPOR5,14,Y,DESCANSO,,,,,,,,,,,,


---
# Exportar CSV
---

In [32]:
df_modelos_eu.to_csv('resultados/db_models.csv', index = False)

In [33]:
df_maestro_patrones.to_csv('resultados/rotation_pattern_masters.csv', index = False)

In [34]:
df_detalle_patrones.to_csv('resultados/rotation_pattern_details.csv', index = False)

---
# Validaciones para verificar que todo existe en WFS
---
Si algo no existe se debe crear en WFS antes se subir cualquier archivo.

## Estaciones

In [35]:
min_estaciones = df_modelos_eu['StationName'].dropna().drop_duplicates()
estaciones_faltantes = min_estaciones[ ~min_estaciones.isin(v_estaciones) ]

estaciones_faltantes.to_excel('/content/estaciones_faltantes.xlsx', index = False)
estaciones_faltantes

7     SS_INDUS_PREP_FISI
59        SS_INDUS_ALGES
Name: StationName, dtype: object

## Turnos

In [36]:
min_turnos = df_modelos_eu['ShiftName'].dropna().drop_duplicates()
turnos_faltantes = min_turnos[ ~min_turnos.isin(v_turnos) ]

turnos_faltantes.to_excel('/content/turnos_faltantes.xlsx', index = False)
turnos_faltantes

0      IPS_0600_0700
1      IPS_0600_0800
3      IPS_0600_0900
41     IPS_0600_1900
45     IPS_0600_2000
51     IPS_0600_2100
53     IPS_0700_0800
54     IPS_0700_0900
120    IPS_0700_2000
124    IPS_0700_2100
125    IPS_0700_2200
152    IPS_0900_1100
156    IPS_0900_1400
163    IPS_0900_2200
164    IPS_1000_1600
170    IPS_1100_1200
172    IPS_1100_1300
173    IPS_1100_1700
174    IPS_1100_1800
182    IPS_1100_2200
183    IPS_1200_1300
191    IPS_1300_1400
194    IPS_1300_1500
195    IPS_1300_1600
226    IPS_1400_1600
228    IPS_1400_1700
248    IPS_1500_1700
257    IPS_1500_2200
259    IPS_1600_1700
260    IPS_1600_1800
269    IPS_1700_1800
270    IPS_1700_1900
272    IPS_1700_2000
275    IPS_1700_2200
276    IPS_1800_2000
277    IPS_1900_2100
278    IPS_1900_2200
280    IPS_2000_2100
281    IPS_2000_2200
282    IPS_2100_2200
Name: ShiftName, dtype: object

## Cualificaciones

In [37]:
min_cualificaciones = df_modelos_eu['QualificationNames'].dropna().drop_duplicates()
cualificaciones_faltantes = min_cualificaciones[ ~min_cualificaciones.isin(v_cualificaciones) ]

cualificaciones_faltantes.to_excel('/content/cualificaciones_faltantes.xlsx', index = False)
cualificaciones_faltantes

0       IPS_DERMATOLO
1       IPS_NUTRICION
2         IPS_ORTOPED
4            IPS_ENDO
5          IPS_GINECO
6         IPS_HIGIENE
7     IPS_MED_DEPORTO
8      IPS_ODONT_GRAL
15       IPS_OTORRINO
16     IPS_PSICOLOGIA
22       IPS_FISIATRA
27       IPS_MED_GRAL
48      IPS_ODONT_ESP
54       IPS_PEDIATRA
56          IPS_ORTOD
59      IPS_MEDIDOLOR
65         IPS_PERIOD
66      IPS_REHA_ORAL
67    IPS_CIRUGIGENER
75        IPS_OFTALMO
79       IPS_UROLOGIA
87    IPS_ODONTOPEDIA
97    IPS_TRABASOCIAL
Name: QualificationNames, dtype: object